# Worksheet

[FORM](https://forms.gle/EqSpaVy88W9sZBH2A)

## Support Vector Machines

Implement the perceptron algorithm to create an animation of the algorithm. Upload the gif to the form.

In [8]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

TEMPFILE = "temp.png"
CENTERS = [[0, 1], [1, 0]]

# Dataset
X, labels = datasets.make_blobs(n_samples=10, centers=CENTERS, cluster_std=0.2, random_state=0)
Y = np.array(list(map(lambda x : -1 if x == 0 else 1, labels.tolist())))

# Initializing w and b
w = np.array([1, 1])
b = 0.1

# Perceptron Parameters
epochs = 200
alpha = .10
expanding_rate = .99
retracting_rate = 1.1

def snap(x, w, b, error):
    """
        Plot the street induced by w and b.
        Circle the point x in red if it was
        misclassified or in yellow if it was
        classified correctly.
    """

    xplot = np.linspace(-3, 3)
    cs = np.array([x for x in 'gb'])

    svm = (-w[0] * xplot - b) / w[1]    
    left_svm = (-w[0] * xplot - b - 1) / w[1]
    right_svm = (-w[0] * xplot - b + 1) / w[1]

    fig, ax = plt.subplots()
    ax.scatter(X[:,0],X[:,1],color=cs[labels].tolist(), s=50, alpha=0.8)
    if error:
        ax.add_patch(plt.Circle((x[0], x[1]), .2, color='r',fill=False))
    else:
        ax.add_patch(plt.Circle((x[0], x[1]), .2, color='y',fill=False))
    ax.plot(xplot, left_svm, 'g--', lw=2)
    ax.plot(xplot, svm, 'r-', lw=2)
    ax.plot(xplot, right_svm, 'b--', lw=2)
    ax.set_xlim(min(X[:, 0]) - 1, max(X[:,0]) + 1)
    ax.set_ylim(min(X[:, 1]) - 1, max(X[:,1]) + 1)
    fig.savefig(TEMPFILE)
    plt.close()

    return im.fromarray(np.asarray(im.open(TEMPFILE)))

# hi - alice 
images = []
for _ in range(epochs):
    # pick a point from X at random
    i = np.random.randint(0, len(X))
    x, y = X[i], Y[i]
    error = False

    prediction = np.sign(np.dot(w, x) + b)
    
    # 오분류된 경우 가중치와 편향 업데이트
    if y * prediction <= 0:
        # move in that derection (i.e update w and b)
        error = True
        w = w + alpha * y * x
        b = b + alpha * y
        
        w = w * expanding_rate
        b = b * expanding_rate
    else:
        # if no, are you in the street?
           # if yes - > retract
           # if no - > expand
        margin = y * (np.dot(w, x) + b) 
        if margin < 1:
            w = w * retracting_rate
            b = b * retracting_rate
        else:
            w = w * expanding_rate
            b = b * expanding_rate

    images.append(snap(x, w, b, error))

images[0].save(
    'svm.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=100
)